In [137]:
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import pandas as pd
# import tensorflow as tf
import glob
import pandas as pd
from joblib import dump, load
import json

In [138]:
## loading dataset
df = pd.read_csv('../data/raw/diabetes.csv')

In [139]:
## Cleaning the dataset
filtereddf = df[ (df['Insulin'] > 0) & (df['Glucose'] > 0) ]
filtereddf = filtereddf.head(10)

In [140]:
## transforming pandas dataframe into Dice dataframe
dice_data = dice_ml.Data(dataframe=df, continuous_features=df.drop(columns=['Outcome']).columns.tolist(), outcome_name='Outcome')

## loading model
sklearn_model = load('../models/classifiers/diabetes.sav')

## creating dice model
dice_model = dice_ml.Model(model=sklearn_model, backend='sklearn')
dice_exp = dice_ml.Dice(dice_data, dice_model)

/home/joaorulff/miniconda3/envs/cfsim/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/joaorulff/miniconda3/envs/cfsim/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [141]:
for i in list(filtereddf.index):
    
    ## testing
    query_instance = filtereddf.drop(columns=['Outcome']).loc[[i]]
    cfs = dice_exp.generate_counterfactuals(query_instance, total_CFs=500, desired_class="opposite")
    
    filepath = f'../data/counterfactuals/diabetes/{i}.json'
    with open(filepath, 'w') as f:
        json.dump( json.loads( cfs.to_json() ), f)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.34s/it]


In [135]:
## saving filtered dataset
filtereddf['id'] = filtereddf.index
filtereddf.drop(columns=['Outcome']).to_csv('../data/datasets/test_diabetes.csv', index=False)

In [73]:
## testing
query_instance = df.drop(columns=['Outcome']).iloc[[4]]
cfs = dice_exp.generate_counterfactuals(query_instance, total_CFs=500, desired_class="opposite")
cfs.visualize_as_dataframe(show_only_changes=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.07s/it]

Query instance (original outcome : 1)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,137,40,35,168,43.099998,2.288,33,1



Diverse Counterfactual set (new outcome: 0.0)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,-,64.0,-,-,-,24.2,2.288,-,0.0
1,-,-,-,-,-,0.7,2.288,-,0.0
2,5.0,-,-,-,-,10.1,2.288,27.0,0.0
3,-,83.0,-,53.0,-,44.000000000000014,2.288,-,0.0
4,-,-,-,-,-,25.5,2.395,-,0.0
...,...,...,...,...,...,...,...,...,...
495,-,-,-,-,-,9.8,0.697,-,0.0
496,-,-,-,-,337.0,10.3,0.736,37.0,0.0
497,-,16.0,-,-,101.0,57.6,2.288,-,0.0
498,14.0,-,-,-,318.0,15.1,2.288,-,0.0


In [ ]:
## generating json query instances
test = df.drop(columns=['Outcome']).iloc[[4]].to_json(orient='records')
test_df = pd.read_json(test, orient ='records')